<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 2 - 1.5 Hours </h1>
<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

<b> Group name:</b> # The name of your group
 
 
The aim of this session is to practice with Convolutional Neural Networks. Each group should fill and run appropriate notebook cells. 


Generate your final report (export as HTML) and upload it on the submission website http://bigfoot-m1.eurecom.fr/teachingsub/login (using your deeplearnXX/password). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed and submitted by May 30th 2018 (23:59:59 CET).

# Introduction

In the previous Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%. Can you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks,  **LeNet-5**, to go to more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell below to load the MNIST data that comes with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.layers import flatten
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels
print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

epsilon = 1e-10 # this is a parameter you will use later

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Image Shape: (784,)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


# Section 1 : My First Model in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example
**y=softmax(Wx+b)** seen in the first lab. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the TensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [2]:
#STEP 1

# Parameters
learning_rate = 0.001
training_epochs = 40
batch_size = 128
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

#STEP 2 

# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch:  01   =====> Loss= 2.097411711
Epoch:  02   =====> Loss= 1.764955707
Epoch:  03   =====> Loss= 1.523936709
Epoch:  04   =====> Loss= 1.347516083
Epoch:  05   =====> Loss= 1.215537611
Epoch:  06   =====> Loss= 1.114600551
Epoch:  07   =====> Loss= 1.035305682
Epoch:  08   =====> Loss= 0.971565528
Epoch:  09   =====> Loss= 0.919358478
Epoch:  10   =====> Loss= 0.875591001
Epoch:  11   =====> Loss= 0.838403968
Epoch:  12   =====> Loss= 0.806569218
Epoch:  13   =====> Loss= 0.778790417
Epoch:  14   =====> Loss= 0.754505948
Epoch:  15   =====> Loss= 0.732968346
Epoch:  16   =====> Loss= 0.713635123
Epoch:  17   =====> Loss= 0.696382746
Epoch:  18   =====> Loss= 0.680580249
Epoch:  19   =====> Loss= 0.666418054
Epoch:  20   =====> Loss= 0.653508346
Epoch:  21   =====> Loss= 0.641403125
Epoch:  22   =====> Loss= 0.630445238
Epoch:  23   =====> Loss= 0.619936790
Epoch:  24   =====> Loss= 0.610809653
Epoch:  25   =====> Loss= 0.601670351
Epoch:  26   =====> Loss= 0.593368141
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Open a Terminal and run the command line **"tensorboard --logdir=lab_2/log_files/"**
- Click on "Tensorboard web interface" in Zoe  


Enjoy It !! 


# Section 2 : The 99% MNIST Challenge !

<b> Part 1 </b> : LeNet5 implementation

You are now more familar with **TensorFlow** and **TensorBoard**. In this section, you are to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

Then, you will make some optimizations to get more than 99% of accuracy.

For more informations, have a look at this list of results: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="lenet.png",width="800" height="600" align="center">
<center><span>Figure 1: Lenet-5 </span></center>





The LeNet architecture takes a 28x28xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU. **MaxPooling:** The output shape should be 14x14x6. 

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU. **MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected:** This should have 10 outputs. **Activation:** softmax.


<b> Question 2.1.1 </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


In [2]:
# Functions for weigths and bias initilization 
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0., shape=shape)
  return tf.Variable(initial)

In [4]:
def LeNet5_Model(x):    
    # your inmplementation goes here
    # Input Layer
    input_layer = tf.reshape(x, [-1, 28, 28, 1])
    # Layer 1 - Convolution (5x5): The output shape should be 28x28x6. Activation: ReLU
    conv1_w=weight_variable(shape = [5,5,1,6])
    conv1_b=bias_variable(shape=[6])
    conv1 = tf.nn.conv2d(input_layer,conv1_w,strides = [1,1,1,1],padding="SAME")+conv1_b
    conv1 = tf.nn.relu(conv1)
     # Pooling Layer # MaxPooling: The output shape should be 14x14x6.
    pool_1 = tf.nn.max_pool(conv1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    # Layer 2 - Convolution (5x5): The output shape should be 10x10x16. Activation: ReLU.
    conv2_w=weight_variable(shape=[5,5,6,16])
    conv2_b=bias_variable(shape=[16])
    conv2 = tf.nn.conv2d(pool_1,conv2_w,strides = [1,1,1,1],padding="VALID")+conv2_b
    conv2=tf.nn.relu(conv2)
    # Pooling Layer # MaxPooling: The output shape should be 5x5x16.
    pool2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
 
    #Flatten:
    pool2_flat = tf.reshape(pool2, [-1, 5*5*16])
    
    # Layer 3 - Fully Connected: This should have 120 outputs. Activation: ReLU.
    fc1_w = weight_variable(shape=[400,120])
    fc1_b = bias_variable(shape=[120])
    fc1 = tf.matmul(pool2_flat,fc1_w) + fc1_b
    fc1=tf.nn.relu(fc1)  
    # Layer 4 - Fully Connected: This should have 84 outputs. Activation: ReLU.
    fc2_w = weight_variable(shape=[120,84])
    fc2_b = bias_variable(shape=[84])
    
    
    fc2= tf.matmul(fc1,fc2_w) + fc2_b
    fc2=tf.nn.relu(fc2)    
    # Layer 5 - Fully Connected: This should have 10 outputs. Activation: softmax.
    fc3_w = weight_variable(shape=[84,10])
    fc3_b = bias_variable(shape=[10])
    fc3 = tf.matmul(fc2,fc3_w) + fc3_b

   # fc3=tf.nn.softmax(fc3)                            
    return fc3
    

<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 

* **1st convolutional layer:** the number of parameters is: number of weights=KxLxDxF and  number biases=F where k is the height of the filter and L is the width and D is the depth and F is the number of filters so in the firsr layer we have: weights=5x5x1x6=150 and biases=6
* **2nd convolutional layer:** weights=5x5x16=400 and biases=16
* **1st Fully connected layer:** the number parmaters is: weights=nxm where n is the of inputs and m is the number of outputs and biases=m so we have: weights=5*5*16*120=48000 and biases=120
* **2nd fully connected layer:** weights=48120*84=4,042,080 and biases=84
* **3rd fully connected layer:** weights=4,042,164*10=40,421,640 and biases=10
* **Total number of parameters:** 150+6+48,000+120+4,042,080+84+40,421,640+10=44,512,090

<b> Question 2.1.3. </b>  Define your model, its accuracy and the loss function according to the following parameters (you can look at Section 1 to see what is expected):

     Learning rate: 0.001
     Loss Fucntion: Cross-entropy
     Optimizer: tf.train.GradientDescentOptimizer
     Number of epochs: 40
     Batch size: 128

In [5]:
tf.reset_default_graph() # reset the default graph before defining a new model

# Parameters
init_learning_rate = 0.001
training_epochs = 1000
batch_size =128 
logs_path = 'log_files/'

# Model, loss function and accuracy
x = tf.placeholder(tf.float32, [None, 784], name='InputData')

y = tf.placeholder(tf.float32, [None,10], name='labels')
z= tf.placeholder(tf.float32, [None,10], name='predictions')
with tf.name_scope('Model'):
    # Model
    logits=LeNet5_Model(x)
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits))
with tf.name_scope('SGD'):
    # Gradient Descent
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(init_learning_rate, global_step, 2000, 0.5, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost, global_step=global_step)

with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [6]:
def evaluate(logits, labels):
    # logits will be the outputs of your model, labels will be one-hot vectors corresponding to the actual labels
    # logits and labels are numpy arrays
    # this function should return the accuracy of your model
  
    return np.sum(np.argmax(logits, 1) == np.argmax(labels,1)) / logits.shape[0]
     

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 



In [10]:

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss_LeNet-5_SGD", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_LeNet-5_SGD", acc)
# Create a summary to monitor accuracy tensor
#tf.summary.scalar("Accuracy_LeNet-5_SGD_on_evalution", acc1)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Initializing the variables
def train(init, sess, logs_path, n_epochs, batch_size, optimizer, cost, merged_summary_op):
    # optimizer and cost are the same kinds of objects as in Section 1
    # Train your model
    sess.run(init)
    display_step=1
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    for epoch in range(n_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=True)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary,train_accuracy = sess.run([optimizer, cost, merged_summary_op,acc],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
            # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))
            # Print the accuracy on testing data
            # Calculate accuracy
          #  predictions=sess.run(logits,feed_dict={x :mnist.train.images})
            
            print("Training Accuracy:", train_accuracy)
            predictions=sess.run(logits,feed_dict={x :mnist.validation.images})
            print("Validation Accuracy:", evaluate(predictions,mnist.validation.labels))
            
           
    print("Optimization Finished!")
    #Save the model after Training
    saver = tf.train.Saver()
    summary_writer.flush()
        
with tf.Session() as sess:
    train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)

Epoch:  01   =====> Loss= 2.288331872
Training Accuracy: 0.09375
Validation Accuracy: 0.1102
Epoch:  02   =====> Loss= 2.253393638
Training Accuracy: 0.1484375
Validation Accuracy: 0.221
Epoch:  03   =====> Loss= 2.195958021
Training Accuracy: 0.390625
Validation Accuracy: 0.3744
Epoch:  04   =====> Loss= 2.068163616
Training Accuracy: 0.5625
Validation Accuracy: 0.5428
Epoch:  05   =====> Loss= 1.760897916
Training Accuracy: 0.609375
Validation Accuracy: 0.6458
Epoch:  06   =====> Loss= 1.433676911
Training Accuracy: 0.734375
Validation Accuracy: 0.7228


KeyboardInterrupt: 

<b> Question 2.1.6 </b> : Use TensorBoard to visualise and save loss and accuracy curves. 
You will save figures in the folder **"lab_2/MNIST_figures"** and display them in your notebook.

In [ ]:
tf.extract_image_patches()

Please put your loss and accuracy curves here.

<b> Part 2 </b> : LeNET 5 Optimization

<b> Question 2.2.1 </b>

- Retrain your network with AdamOptimizer 

In [8]:
tf.reset_default_graph() # reset the default graph before defining a new model

# Parameters
init_learning_rate = 0.001
training_epochs = 1000
batch_size =128 
logs_path = 'log_files/'

# Model, loss function and accuracy
x = tf.placeholder(tf.float32, [None, 784], name='InputData')

y = tf.placeholder(tf.float32, [None,10], name='labels')
with tf.name_scope('Model'):
    # Model
    logits=LeNet5_Model(x)
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits))
with tf.name_scope('SGD'):
    # Gradient Descent
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(init_learning_rate, global_step, 2000, 0.5, staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost, global_step=global_step)

with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))
tf.reset_default_graph() # reset the default graph before defining a new model

# Parameters
init_learning_rate = 0.001
training_epochs = 1000
batch_size =128 
logs_path = 'log_files/'

# Model, loss function and accuracy
x = tf.placeholder(tf.float32, [None, 784], name='InputData')

y = tf.placeholder(tf.float32, [None,10], name='labels')
z= tf.placeholder(tf.float32, [None,10], name='predictions')

with tf.name_scope('Model'):
    # Model
    logits=LeNet5_Model(x)
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits))
with tf.name_scope('SGD'):
    # Gradient Descent
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(init_learning_rate, global_step, 2000, 0.5, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost, global_step=global_step)

with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))


In [9]:
# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss_LeNet-5_SGD", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_LeNet-5_SGD", acc)
# Create a summary to monitor accuracy tensor
#tf.summary.scalar("Accuracy_LeNet-5_SGD_on_evalution", acc1)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Initializing the variables
def train(init, sess, logs_path, n_epochs, batch_size, optimizer, cost, merged_summary_op):
    # optimizer and cost are the same kinds of objects as in Section 1
    # Train your model
    sess.run(init)
    display_step=100
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    for epoch in range(n_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=True)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary,train_accuracy = sess.run([optimizer, cost, merged_summary_op,acc],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
            # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))
            # Print the accuracy on testing data
            # Calculate accuracy
          #  predictions=sess.run(logits,feed_dict={x :mnist.train.images})
            
            print("Training Accuracy:", train_accuracy)
            predictions=sess.run(logits,feed_dict={x :mnist.test.images})
            print("Validation Accuracy:", evaluate(predictions,mnist.test.labels))
            
           
    print("Optimization Finished!")
    #Save the model after Training
    saver = tf.train.Saver()
    summary_writer.flush()
        
with tf.Session() as sess:
    train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)

Epoch:  01   =====> Loss= 2.291449495
Training Accuracy: 0.15625
Validation Accuracy: 0.1442
Epoch:  02   =====> Loss= 2.261923220
Training Accuracy: 0.25
Validation Accuracy: 0.2652
Epoch:  03   =====> Loss= 2.216270101
Training Accuracy: 0.4140625
Validation Accuracy: 0.4109
Epoch:  04   =====> Loss= 2.121105467
Training Accuracy: 0.5
Validation Accuracy: 0.5236
Epoch:  05   =====> Loss= 1.891155061
Training Accuracy: 0.5703125
Validation Accuracy: 0.6099
Epoch:  06   =====> Loss= 1.602287330
Training Accuracy: 0.6953125
Validation Accuracy: 0.6822
Epoch:  07   =====> Loss= 1.283147906
Training Accuracy: 0.671875
Validation Accuracy: 0.7424
Epoch:  08   =====> Loss= 0.997380043
Training Accuracy: 0.75
Validation Accuracy: 0.794
Epoch:  09   =====> Loss= 0.793482465
Training Accuracy: 0.8125
Validation Accuracy: 0.8238
Epoch:  10   =====> Loss= 0.671455082
Training Accuracy: 0.84375
Validation Accuracy: 0.84
Epoch:  11   =====> Loss= 0.615836185
Training Accuracy: 0.8671875
Validation

KeyboardInterrupt: 


- fill the table above:


| Optimizer            |  Gradient Descent  |    AdamOptimizer    |
|----------------------|--------------------|---------------------|
| Testing Accuracy     |         ???        |        ???          |       
| Training Time        |         ???        |        ???          |  

- Which optimizer gives the best accuracy on test data?

**Your answer:** ...


<b> Question 2.2.2</b> Try to add dropout (keep_prob = 0.75) before the first fully connected layer. You will use tf.nn.dropout for that purpose. What accuracy do you achieve on testing data?

**Accuracy achieved on testing data:** ...

In [38]:
def LeNet5_Model_Dropout(image):    
  
    # your inmplementation goes here
    # Input Layer
    input_layer = tf.reshape(x, [-1, 28, 28, 1])
    # Layer 1 - Convolution (5x5): The output shape should be 28x28x6. Activation: ReLU
    conv1_w=weight_variable(shape = [5,5,1,6])
    conv1_b=bias_variable(shape=[6])
    conv1 = tf.nn.conv2d(input_layer,conv1_w,strides = [1,1,1,1],padding="SAME")+conv1_b
    conv1 = tf.nn.relu(conv1)
     # Pooling Layer # MaxPooling: The output shape should be 14x14x6.
    pool_1 = tf.nn.max_pool(conv1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    # Layer 2 - Convolution (5x5): The output shape should be 10x10x16. Activation: ReLU.
    conv2_w=weight_variable(shape=[5,5,6,16])
    conv2_b=bias_variable(shape=[16])
    conv2 = tf.nn.conv2d(pool_1,conv2_w,strides = [1,1,1,1],padding="VALID")+conv2_b
    conv2=tf.nn.relu(conv2)
    # Pooling Layer # MaxPooling: The output shape should be 5x5x16.
    pool2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
 
    #Flatten:
    pool2_flat = tf.reshape(pool2, [-1, 5*5*16])
    fc1_drop = tf.nn.dropout(fc1, keep_prob = 0.75)
    # Layer 3 - Fully Connected: This should have 120 outputs. Activation: ReLU.
    fc1_w = weight_variable(shape=[400,120])
    fc1_b = bias_variable(shape=[120])
    fc1 = tf.matmul(fc1_drop,fc1_w) + fc1_b
    fc1=tf.nn.relu(fc1)  
    # Layer 4 - Fully Connected: This should have 84 outputs. Activation: ReLU.
    fc2_w = weight_variable(shape=[120,84])
    fc2_b = bias_variable(shape=[84])
    
    
    fc2= tf.matmul(fc1,fc2_w) + fc2_b
    fc2=tf.nn.relu(fc2)    
    # Layer 5 - Fully Connected: This should have 10 outputs. Activation: softmax.
    fc3_w = weight_variable(shape=[84,10])
    fc3_b = bias_variable(shape=[10])
    fc3 = tf.matmul(fc2,fc3_w) + fc3_b
                       
    return fc3
    
     


In [ ]:
tf.reset_default_graph()
# your implementation goes here

# Parameters
init_learning_rate = 0.001
training_epochs = 1000
batch_size =128 
logs_path = 'log_files/'

# Model, loss function and accuracy
x = tf.placeholder(tf.float32, [None, 784], name='InputData')

y = tf.placeholder(tf.float32, [None,10], name='labels')
z= tf.placeholder(tf.float32, [None,10], name='predictions')
with tf.name_scope('Model'):
    # Model
    logits=LeNet5_Model_Dropout(x)
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits))
with tf.name_scope('SGD'):
    # Gradient Descent
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(init_learning_rate, global_step, 2000, 0.5, staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost, global_step=global_step)

with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))


# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss_LeNet-5_SGD", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_LeNet-5_SGD", acc)
# Create a summary to monitor accuracy tensor
#tf.summary.scalar("Accuracy_LeNet-5_SGD_on_evalution", acc1)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Initializing the variables
def train(init, sess, logs_path, n_epochs, batch_size, optimizer, cost, merged_summary_op):
    # optimizer and cost are the same kinds of objects as in Section 1
    # Train your model
    sess.run(init)
    display_step=100
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    for epoch in range(n_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=True)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary,train_accuracy = sess.run([optimizer, cost, merged_summary_op,acc],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
            # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))
            # Print the accuracy on testing data
            # Calculate accuracy
          #  predictions=sess.run(logits,feed_dict={x :mnist.train.images})
            
            print("Training Accuracy:", train_accuracy)
            predictions=sess.run(logits,feed_dict={x :mnist.test.images})
            print("Validation Accuracy:", evaluate(predictions,mnist.test.labels))
            
           
    print("Optimization Finished!")
    #Save the model after Training
    saver = tf.train.Saver()
    summary_writer.flush()
        
with tf.Session() as sess:
    train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)

